docker run -it --link trdgame_db_1  python:3.5 bash

In [ ]:
import pandas as pd

In [ ]:
from pymongo import MongoClient

# connection

In [ ]:
mongo_host = 'trdgame_db_1'
mongo_db = 'api'

In [ ]:
client = MongoClient(mongo_host, 27017)
client.server_info()

In [ ]:
client.database_names()

In [ ]:
db = client[mongo_db] #flask app

In [ ]:
db.collection_names()

# get raw data

## games

In [ ]:
players_games = db['players_games']
players_games.count()

In [ ]:
list_players_games = []
cursor = players_games.find({})
for doc in cursor:
    list_players_games.append(doc)
len(list_players_games)

### games pre-processing

In [ ]:
bridge_gameid_playerid = {}

# unwrap objects
obj_header = ['game', 'player', 'portfolio', 'server']
obj_second_level_header = ['portfolio_positions']
key_cleaning = ['game_currentRound', 'game_gameOver', 'game_numberPlayers', 
                'player_omsOption', 'player_ranking', 'player_rankingChg', 'player_rankingPrev', 'player_textTrader', 
               'portfolio_positions_derivatives']
for doc in list_players_games:
    
    del doc['executedOrders']
    del doc['pendingOrders']
    
    for header in obj_header:
        for key in doc[header]:
            doc[header + '_' + key] = doc[header][key]
        del doc[header]
    
    for header in obj_second_level_header:
        for key in doc[header]:
            doc[header + '_' + key] = doc[header][key]
        del doc[header]
    
    for key in key_cleaning:
        del doc[key]
    
    bridge_gameid_playerid[ str(doc['_id']) ] = doc['player_name']
    
    doc['_id'] = str( doc['_id'] ) #mongo-ObjectId

list_players_games[0]

In [ ]:
# df_players_games = pd.DataFrame(list(players_games.find())) # direct one line
df_players_games = pd.DataFrame(list_players_games)
df_players_games

## orders

In [ ]:
orders = db['orders']
orders.count()

In [ ]:
list_orders = []
cursor = orders.find({})
for doc in cursor:
    list_orders.append(doc)
len(list_orders)

### orders pre-processing

In [ ]:
# unwrap objects
dict_orders = {}
obj_header = ['mkcontext', 'prtcontext']
key_cleaning = []
for doc in list_orders:
    
    for header in obj_header:
        for key in doc[header]:
            doc[header + '_' + key] = doc[header][key]
        del doc[header]
    
    for key in key_cleaning:
        del doc[key]
    
    doc['player_name'] = bridge_gameid_playerid[ doc['game_id'] ]
    
    dict_orders[ str(doc['_id']) ] = doc
    
    doc['_id'] = str( doc['_id'] )

list_orders[0]

In [ ]:
#df_orders = pd.DataFrame(list(orders.find())) # direct one line
df_orders = pd.DataFrame(list_orders)
df_orders.head()

## executions

In [ ]:
executions = db['executions']
executions.count()

In [ ]:
list_executions = []
cursor = executions.find({})
for doc in cursor:
    list_executions.append(doc)
len(list_executions)

### executions pre-processing

In [ ]:
# unwrap objects
obj_header = ['mkcontext', 'prtcontext']
key_cleaning = ['ordertimestamp', 'ordertype', 'orderprice']
for doc in list_executions:
    
    for header in obj_header:
        for key in doc[header]:
            doc[header + '_' + key] = doc[header][key]
        del doc[header]
    
    doc['player_name'] = bridge_gameid_playerid[ doc['game_id'] ]
    
    if 'orderid' in doc:
        order = dict_orders[ doc['orderid'] ]
        for key in order:
            doc['order_' + key] = order[key]
        
        del doc['orderid']
        #doc['order_id'] = str( doc['order__id'] )
        #del doc['order__id']
    
    for key in key_cleaning:
        del doc[key]
    
    doc['_id'] = str( doc['_id'] )
            

list_executions[1]

In [ ]:
#df_executions = pd.DataFrame(list(executions.find())) # direct one line
df_executions = pd.DataFrame(list_executions)
df_executions.head()

## surveys

In [ ]:
surveys = db['surveys']
surveys.count()

In [ ]:
list_surveys = []
cursor = surveys.find({})
for doc in cursor:
    list_surveys.append(doc)
len(list_surveys)

### surveys pre-processing

In [ ]:
obj_header = ['response', 'prtcontext', 'mkcontext']
for doc in list_surveys:
    doc['prtcontext'] = doc['submittedContent']['prtcontext']
    del doc['submittedContent']['prtcontext']
    
    doc['mkcontext'] = doc['submittedContent']['mkcontext']
    del doc['submittedContent']['mkcontext']
    
    doc['game_id'] = doc['submittedContent']['game_id']
    del doc['submittedContent']['game_id']
    
    doc['player_name'] = bridge_gameid_playerid[ doc['game_id'] ]
    
    for header in obj_header:
        for key in doc[header]:
            doc[header + '_' + key] = doc[header][key]
        del doc[header]
    
    doc['_id'] = str( doc['_id'] )

list_surveys[0]

In [ ]:
#df_surveys = pd.DataFrame(list(surveys.find())) # direct one line
df_surveys = pd.DataFrame(list_surveys)
df_surveys

In [ ]:
df_surveys[df_surveys['name'] == 'risks']

In [ ]:
df_surveys[df_surveys['name'] == 'derivatives']

# exports

In [ ]:
df_players_games.to_json('players_games.txt', orient='records')

In [ ]:
df_orders.to_json('orders.txt', orient='records')

In [ ]:
df_executions.to_json('executions.txt', orient='records')

In [ ]:
df_surveys.to_json('surveys.txt', orient='records')